In [1]:
import Pkg
Pkg.add("CSV")
Pkg.update()
Pkg.add("DataFrames")
Pkg.add("DataStructures")
Pkg.add("TextAnalysis")
using DataFrames
using CSV
using TextAnalysis, DataStructures
import Dates
#Pkg.add(Pkg.PackageSpec(url="https://github.com/JuliaComputing/JuliaAcademyData.jl"))
#using JuliaAcademyData; activate("Parallel_Computing")

In [2]:
startTime = Dates.Time(Dates.now()) 

df = DataFrame(CSV.File("/home/joelkik/HPC/Project/Data/amazon_reviews_us_Apparel_v1_00.tsv"))
mylock = ReentrantLock()
docs=[]
Threads.@threads for i in 1:nrow(df)
    if typeof(df.review_body[i])==String
        sd=TokenDocument(df.review_body[i])
        prepare!(sd, strip_stopwords)
        prepare!(sd, strip_non_letters)
        lock(mylock)
        push!(docs, sd) 
        unlock(mylock)
    end
end

┌ Warning: thread = 6 warning: parsed expected 15 columns, but didn't reach end of line around data row: 2187736. Parsing extra columns and widening final columnset
└ @ CSV /home/joelkik/.julia/packages/CSV/jFiCn/src/file.jl:579
┌ Warning: thread = 8 warning: only found 14 / 15 columns around data row: 2888568. Filling remaining columns with `missing`
└ @ CSV /home/joelkik/.julia/packages/CSV/jFiCn/src/file.jl:578
┌ Warning: thread = 7 warning: only found 14 / 15 columns around data row: 4349503. Filling remaining columns with `missing`
└ @ CSV /home/joelkik/.julia/packages/CSV/jFiCn/src/file.jl:578
┌ Warning: thread = 1 warning: parsed expected 15 columns, but didn't reach end of line around data row: 726030. Parsing extra columns and widening final columnset
└ @ CSV /home/joelkik/.julia/packages/CSV/jFiCn/src/file.jl:579
┌ Warning: thread = 2 warning: only found 14 / 15 columns around data row: 1546812. Filling remaining columns with `missing`
└ @ CSV /home/joelkik/.julia/packages/CS

In [3]:
length(docs)

5881177

In [4]:
termDict=Dict()
docTerm=Dict()
docTermCount=[]
for i in 1:length(docs)
    dt=Dict()
    for t in docs[i].tokens
        if haskey(termDict, t)
            termDict[t]=termDict[t]+1
        else
            termDict[t]=1
        end
        if haskey(docTerm, t)
            push!(docTerm[t], i)
        else
            docTerm[t]=Set(i)
        end
        if haskey(dt, t)
            dt[t]=dt[t]+1
        else
            dt[t]=1
        end
    end
    push!(docTermCount, dt)
end

In [5]:
IDF=Dict()
for (k,v) in docTerm
    IDF[k]=log2(length(docs)/length(v))
end

In [6]:
tf_idf=[]
mylock = ReentrantLock()
Threads.@threads for i in 1:length(docs)
    dt=Dict()
    for (k,v) in docTermCount[i]
        dt[k]=v*IDF[k]
    end
    lock(mylock)
    push!(tf_idf, dt)
    unlock(mylock)
end

In [7]:
endTime = Dates.Time(Dates.now()) 

00:45:52.309

In [8]:
println("Time Taken: ", endTime-startTime)

Time Taken: 885829000000 nanoseconds
